In [1]:
!git clone https://github.com/hung20gg/grade_func.git
!cd grade_func && git pull

fatal: destination path 'grade_func' already exists and is not an empty directory.


Already up to date.


In [2]:
from grade_func.grader import test_func_from_file

### Ex1: Write a function, using regex to validate the datetime format (1.5 point)
Given a datetime as a string, check does the datetime satisfy the condition DD-MM-YYYY or DD/MM/YYYY. Return True or False

In [3]:
import re
patterns=[]
def validate_datetime(date : str) -> bool:
    #DD/MM/YYYY
    patterns.append(r'(0[1-9]|[12][0-9]|3[01])[\/-](0[13578]|1[02])[\/-][1-9](\d{3})$')
    patterns.append(r'(0[1-9]|[12][0-9]|30)[\/-](0[469]|11)[\/-][1-9](\d{3})$')
    patterns.append(r'(29)[\/-](02)[\/-]\d{2}(0[48]|[13579][26]|[2468][048])')
    patterns.append(r'(0[1-9]|1[0-9]|2[0-8])[\/-](02)[\/-][1-9](\d{3})$')

    for pattern in patterns:
        if re.match(pattern,date):
            return True
    return False

validate_datetime("07/07/1927")

True

In [4]:
q1 = test_func_from_file(validate_datetime,'hw9_q_1.txt',plain_text_input=True, max_grade=1.5)
print("Point for Question 1 is", q1)

Total tests: 19
Passed tests: 19
Point for Question 1 is 1.5


### Ex2: Sum of all element (1.5 point)
Find the sum of all numerical element passed into the function

In [5]:
def sum_(*nums) -> int:
    return sum(x for x in nums)

In [6]:
q2 = test_func_from_file(sum_,'hw9_q_2.txt', max_grade=1.5)
print("Point for Question 2 is", q2)

Total tests: 19
Passed tests: 19
Point for Question 2 is 1.5


### Ex3: Dynamic function with **kwargs (2 point)

You are given 2 function:

$f(x_1, x_2, x_3, x_4) = x_1 + x_2^2 + x_3^2 + x_4$

$g(x_1, x_2) = x_1 + x_2$


You must write function f and g to satisfy the router below

In [22]:
def f(x1,x2,x3,x4):
    return x1+x2**2+x3**2+x4

def g(x1,x2):
    return x1+x2

# Evaluate func
def router(pick, x1, x2, x3, x4):
    def your_func(pick, **kwargs):
        if pick == 'f':
            return f(x1,x2,x3,x4)
        elif pick == 'g':
            return g(x1,x2)
    return your_func(pick, x1=x1, x2=x2, x3=x3, x4=x4)

router('f', 170809, -10776, 149550, -57367)

22481438118

In [23]:
q3 = test_func_from_file(router, 'hw9_q_3.txt', max_grade=2)
print("Point for Question 3 is", q3)

Total tests: 19
Passed tests: 19
Point for Question 3 is 2.0


### Ex4: Vectorize a sentence with Word2Vec (5 point)

For simplicity, we will ignore how Word2Vec was trained.

Word2Vec is a hashmap (dictionary) with keys are the words and values are the corresponding vectors (50-dim) to the word. To encode a sentence to a vector, you simply get the mean vector of words in the sentence.

Eg.
```
model = {
    "I": [0,1,2],
    "love":[1.5,2,0.3],
    "you":[0.5,1,2]
}
```
The encoding for sentence *I love you* is the mean value of `model["I"]`, `model["love"]`, "model["you"]`
```
vector = [2/3, 4/3, 4.3/3] 
```

In [9]:
!pip install gensim

In [10]:
import gensim.downloader as api
# Load the pre-trained Word2Vec model
model = api.load('glove-wiki-gigaword-50') # Here is your word2vec database

#### Encode a sentence (2 point)

In [24]:
import re
import numpy as np
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    return ' '.join(text.split()).strip()

def get_vector(sentence: str) -> np.ndarray:
    cleaned_sentence = clean_text(sentence)
    words = cleaned_sentence.split()
    word_vectors = []
    
    for word in words:
        if word in model.key_to_index:
            word_vectors.append(model[word])
    
    if not word_vectors:
        print("Warning: No words found in the model.")
        return np.zeros(model.vector_size)

    sentence_vector = np.mean(word_vectors, axis=0)
    
    return sentence_vector


sentence = "i love you"
vector = get_vector(sentence)
vector

array([-0.00701397,  0.54196334, -0.19225435, -0.5246567 ,  0.7782766 ,
       -0.04985666, -0.31802335,  0.1746592 , -0.52432996,  0.4764192 ,
       -0.33454332,  0.93489003, -0.61863667, -0.164186  ,  1.1000067 ,
        0.33991334,  0.29203   ,  0.35769334,  0.07931166, -0.724163  ,
       -0.42256665,  0.87211996,  0.7086167 ,  0.45412335,  1.2277    ,
       -2.0613    , -1.3180667 ,  0.23561667,  1.2105001 , -1.2606801 ,
        3.3339665 ,  0.7460467 , -0.60946995,  0.23688667, -0.31138667,
       -0.179042  ,  0.17087667,  0.119286  ,  0.3511467 , -0.56632334,
        0.09226224, -0.03197267, -0.20612   ,  0.41710332,  0.168862  ,
        0.18619333,  0.08125467, -0.8010633 , -0.20057969,  0.78086996],
      dtype=float32)

### Calculate similarity of 2 sentence

In [12]:
def cosine_similarity(text1: str, text2: str) -> float:
    
    vector1 = get_vector(text1)
    vector2 = get_vector(text2)
    
    if np.all(vector1 == 0) or np.all(vector2 == 0):
        print("Warning: One or both vectors are zero vectors.")
        return 0.0
    
    cosine_sim = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    
    return cosine_sim

sentence1 = "I love you oppa"
sentence2 = "I love you oppa"
similarity_score = cosine_similarity(sentence1, sentence2)
print(f"Cosine Similarity: {similarity_score}")

Cosine Similarity: 1.0000001192092896


### Get top-k similarity (2 point)

Given a list of sentences (knowledge database) and a random sentence, return index of top-k similar sentence in the knowledge database

In [13]:
with open("knowledge.txt") as f:
    database = f.readlines()
database

['I live in the poor countryside with hundreds of hardships\n',
 'Wanting to move to the city, I have to strive so my stomach is always full\n',
 'Then I met you, the broken pieces linger, and at night I miss you\n',
 'In my dreams, I call your name, oh dear love\n',
 'Can you stay here?\n',
 "Do you know that outside, there's a storm?\n",
 'There’s so much loneliness, oh dear love\n',
 'I just wish for you to have peace\n',
 'Holding hands tightly, pressing lips together\n',
 'Sitting on the hillside\n',
 'Who do you love, who are you cherishing?\n',
 'Or are you just alone?\n',
 'Waiting for the future\n',
 'Why not love right now?\n',
 'Give me your innocence\n',
 'Tonight, I will dream\n',
 'Lonely like a poet\n',
 'Sitting alone, lost in thought\n',
 'One person leaves, while the other stays\n',
 'Yet the heart remains confused\n',
 'The sky where we once shared sweet moments\n',
 'Gently holding hands\n',
 'Your dress lightly flutters\n',
 'Sitting on the hillside again\n',
 'You

In [29]:
def top_k_similar(text: str, k: int) -> list:
    text = clean_text(text)
    similarity = []
    for index, line in enumerate(database):
        line = line.strip()
        similarity.append((index, cosine_similarity(line, text)))

    similarity.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in similarity[:k]]


In [30]:
top_k_similar("dolphins are gay sharks",6)

[24, 11, 13, 10, 18, 5]